<a href="https://colab.research.google.com/github/mrsamsonn/Crypto_Price_Analysis/blob/main/LSTM_Crypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing dependencies
#gudie: https://medium.com/geekculture/lstm-for-bitcoin-prediction-in-python-6e2ea7b1e4e4

import pandas as pd
import numpy as np
import math
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from itertools import cycle
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
#install yfinance for pulling data
%pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 485 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [4]:
#import yfinance
import yfinance as yf

#import data to analyze
df = yf.download('BTC-USD')

[*********************100%***********************]  1 of 1 completed


In [5]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2022-11-28,16440.222656,16482.933594,16054.530273,16217.322266,16217.322266,27743025156
2022-11-29,16217.639648,16522.257812,16139.396484,16444.982422,16444.982422,23581685468
2022-11-30,16445.476562,17190.937500,16445.476562,17168.566406,17168.566406,29523576583


In [6]:
#check for missing values
df.isna().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [16]:
#check monthly average open and close

df.index = pd.to_datetime(df.index, format='%Y-%m-%d')
year_2021 = df.loc[(df.index >= '2021-01-01') & (df.index < '2022-01-01')]
year_2021.drop(year_2021[['High','Low','Volume']],axis=1)
months_2021 = year_2021.groupby(year_2021.index.to_series().dt.strftime('%B'))[['Open','Close']].mean()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 
             'September', 'October', 'November', 'December']
months_2021 = round(months_2021.reindex(month_order, axis=0),2)
months_2021 = months_2021.reset_index()

In [18]:
#plot average monthly closing

fig = px.bar(months_2021, x='Date', y=['Open','Close'], barmode='group', title = 'Monthly Average Open & Close Price - Year 2021')
fig.show()